# Estudio inicial de como funciona un LLM
___
### Juan Jose Londoño Cardenas
### Universidad Politécnica de Madrid
### Junio/2025

Como primera medida, haremos Fine Tuning de un modelo LLM preentrenado, con el fin de entender mas su uso al principio. Si se desea conocer desde cero la construccion de un LLM, se recomienda el otro repositorio donde partimos desde cero, en una regresion lineal basica y vamos avanzando hasta un LLM.

- https://github.com/lonj22803/estudio_doctorado_I